In [1]:
# 데이터 사전처리
# 누락 데이터 처리
# 1. 누락 데이터 확인
import pandas as pd
import seaborn as sns

In [2]:
# titanic 데이터 load
df = sns.load_dataset('titanic')
df.info()   # 데이터프레임의 데이터의 인덱스의 갯수와 컬럼별 데이터의 갯수 및 자료 타입

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [50]:
# deck 컬럼이 누락 데이터가 존재하는 것을 확인
# deck 컬럼의 누락데이터가 몇개인지 확인 : value_counts(dropna=False)
print(df.deck.value_counts(dropna=False))  # 컬럼의 유일한 값들의 갯수, Nan 688개가 존재
print(len(df))  # 전체 행의 갯수는 : 891 개

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64
891


In [ ]:
# 누락 데이터를 직접 찾는 방법 : .isnull(), .notnull()
print(df.head().isnull())   # null이면 True, 아니면 False
print(df.head().notnull())  # null이면 False,  아니면 True

# 누락데이터의 갯수 확인
print(df.head().isnull().sum(axis=0))

In [32]:
# isnull(),  value_counts() -> 각 열의 값에 누락변수가 몇개씩 존재하는지 확인
miss_df = df.isnull()   # null이면 True, 아니면 False

for col in miss_df.columns:
    miss_count = miss_df[col].value_counts()  # 각 열의 Nan 갯수
    
    try:
        print("{} : {}".format(col, miss_count[True]))
    except:
        print("{} : {}".format(col, 0))
#         print("error : ", Exception)
    

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


In [36]:
# 누락 데이터 처리 
# Nan 데이터가 500개 이상인 컬럼을 삭제 - deck 688이므로 삭제
df_thresh =df.dropna(axis=1, thresh=500)
df_thresh.columns   # deck 컬럼이 삭제된 자료

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [ ]:
# age 컬럼에 Nan이 있는 인덱스(행)을 제거
df_age = df_thresh.dropna(axis=0, how='any')
print(df_age.columns)
print(df_age.info())

In [ ]:
# 누락 데이터 치환  .fillna(값, inplace=True), fillna(method='ffill' 또는 'bfill')
df['age'].head(10)

# age 의 누락 데이터에 age의 평균값으로 치환
df_age = df.copy()
mean_age = df_age['age'].mean(axis=0)
df_age['age'].fillna(mean_age, inplace=True)

print(df_age['age'].head(10))

# age를 가장 많은 나이로 변경
df_age = df.copy()
mean_age = df_age['age'].max(axis=0)
df_age['age'].fillna(mean_age, inplace=True)

print(df_age['age'].head(10))

In [47]:
# embark_town : 825행부터 829 행까지 정보를 확인
df['embark_town'][825:830]

# 가장 빈번하게 나오는 값으로 대체 
most_data = df['embark_town'].value_counts(dropna=True).idxmax() # dropna=True -> default

df_embark = df.copy()
df_embark['embark_town'].fillna(most_data, inplace=True)
df_embark['embark_town'][825:830]

# 근접한 (이웃) 값으로 대체 
df_embark = df.copy()
df_embark['embark_town'].fillna(method='ffill', inplace=True)
df_embark['embark_town'][825:830]

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object

In [ ]:
df = pd.DataFrame( {'c1': ['a','a','b','a','b'],
                   'c2' : [1,1,1,2,2],
                   'c3': [1,1,2,2,2]})

print(df); print()

# 중복 데이터 확인 .duplicated(), True or False 를 반환 
print(df.duplicated())  # DataFrame의 경우 행 단위의 중복 확인
print(df['c2'].duplicated())  # Series의 경우 컬럼의 중복 확인

# 중복 행 데이터를 제거 : .drop_duplicates()
df2 = df.drop_duplicates()
print(df2); print()

# 컬럼을 기준으로 중복 행 제거
df3 = df.drop_duplicates(subset=['c2','c3'])
print(df3)

In [64]:
# titanic 데이터를 load 해서 
titanic = sns.load_dataset('titanic')
titanic.info()
# age 컬럼이 Nan이면 행을 삭제하고 age 컬럼을 기준으로 중복을 제거한 데이터프레임을 추출
titanic = titanic.dropna(subset=['age'], how='any', axis=0, inplace=True)
titanic.info()

#age 컬럼을 기준으로 중복을 제거한 데이터프레임을 추출
titanic_age =titanic.drop_duplicates(subset=['age'])
titanic_age.info()

# 모든 컬럼의 값이 같은 행을 제거
titanic_dup = titanic.drop_duplicates()
titanic_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data co